In [168]:
from flask import Flask, request
from tensorflow.keras.models import load_model
import numpy as np
import cv2


In [169]:
app = Flask(__name__)
model = load_model('catdog.h5')


In [170]:
classes_list = ['cat','dog']
global label_names
label_names = sorted(classes_list)

In [171]:
def predict(model,image):
  
  image = cv2.imread(image)
  processed_image = preprocess(image)
  results = model.predict(processed_image)
  (x1, y1, x2, y2), confidence = postprocess(image, results)
  return (x1,y1,x2,y2),confidence 

def preprocess(img, image_size = 300):
  
    image =cv2.resize(img, (image_size, image_size))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype("float") / 255.0 
    image = np.expand_dims(image, axis=0) 
    return image

def postprocess(image, results):

    bounding_box, class_probs = results
    h, w = image.shape[:2]
    x1, y1, x2, y2 = bounding_box[0]
    x1 = int(w * x1)
    x2 = int(w * x2)
    y1 = int(h * y1)
    y2 = int(h * y2)
    return  (x1,y1,x2,y2), class_probs

In [172]:
@app.route('/predict', methods = ['POST'])
def home():
    image_path = request.json['image_path']
    bbox, label = predict(model, image_path)
    class_index = np.argmax(label)
    class_label = label_names[class_index]
    conf= str(round(np.max(label),2))
    return {"label" : class_label,"confidence":conf,'bbox':bbox}

In [173]:
if __name__ == '__main__':
  app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


1/1 [==============================] - 3s 3s/step


127.0.0.1 - - [19/Jun/2022 13:42:28] "POST /predict HTTP/1.1" 200 -
